In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline

from fastai.imports import *
from fastai.structured import *
from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display
from sklearn import metrics

### 2 Read Data

In [22]:
PATH = "data/grocery-sales/"

In [23]:
types = {'id': 'int64',
         'item_nbr': 'int32',
         'store_nbr': 'int8',
         'unit_sales': 'float32',
         'onpromotion': 'object'}

In [38]:
%%time
df_all = pd.read_csv(f'{PATH}test.csv', parse_dates=['date'],
                     dtype=types, infer_datetime_format=True, encoding='iso-8859-1')

CPU times: user 3.88 s, sys: 319 ms, total: 4.2 s
Wall time: 3.97 s


In [34]:
!shuf -n 5 -o train.csv

/bin/sh: shuf: command not found


In [35]:
df_all.onpromotion.fillna(False, inplace=True)
df_all.onpromotion = df_all.onpromotion.map({'False': False, 
                                             'True': True})
df_all.onpromotion = df_all.onpromotion.astype(bool)
%time df_all.to_feather('tmp/raw_groceries')

CPU times: user 300 ms, sys: 315 ms, total: 615 ms
Wall time: 799 ms


In [36]:
%time df_all.describe(include='all')

CPU times: user 3.16 s, sys: 190 ms, total: 3.35 s
Wall time: 1.35 s


,id,date,store_nbr,item_nbr,onpromotion
count,3.370464e+06,3370464,3.370464e+06,3.370464e+06,3370464
unique,NaN,16,NaN,NaN,2
top,NaN,2017-08-17,NaN,NaN,False
freq,NaN,210654,NaN,NaN,3171867
mean,1.271823e+08,NaN,2.750000e+01,1.244798e+06,NaN
std,9.729693e+05,NaN,1.558579e+01,5.898362e+05,NaN
min,1.254970e+08,NaN,1.000000e+00,9.699500e+04,NaN
25%,1.263397e+08,NaN,1.400000e+01,8.053210e+05,NaN
50%,1.271823e+08,NaN,2.750000e+01,1.294665e+06,NaN
75%,1.280249e+08,NaN,4.100000e+01,1.730015e+06,NaN


In [ ]:
df_test = pd.read_csv(f'{PATH}test.csv', parse_dates = ['date'],
                      dtype=types, infer_datetime_format=True)
df_test.onpromotion.fillna(False, inplace=True)
df_test.onpromotion = df_test.onpromotion.map({'False': False, 
                                               'True': True})
df_test.onpromotion = df_test.onpromotion.astype(bool)
df_test.describe(include='all')

In [ ]:
df_all.tail()

In [ ]:
df_all.unit_sales = np.log1p(np.clip(df_all.unit_sales, 0, None))

In [ ]:
%time add_datepart(df_all, 'date')

In [ ]:
n_valid = len(df_test)
n_trn = len(df_all) - n_valid
train, valid = split_vals(df_all, n_trn)
train.shape, valid.shape

In [ ]:
%%time
trn, y, nas  = proc_df(train, 'unit_sales')
val, y_val, nas = proc_df(valid, 'unit_sales', nas)

In [ ]:
def rmse(x,y): return math.sqrt(((x-y)**2).mean())
def print_score(m):
    res = [rmse(m.predict(X_train), y_train),
           rmse(m.predict(X_valid), y_valid),
           m.score(X_train, y_train), m.score(X_valid, y_valid)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

In [ ]:
set_rf_samples(1_000_000)

In [ ]:
%time x = np.array(trn, dtype=np.float32)

In [ ]:
m = RandomForestRegressor(n_estimators=20, min_samples_leaf=100, 
                          n_jobs=8)
%time m.fit(x, y)

In [ ]:
%prun m.fit(x, y)

In [ ]:
print_score(m)

In [ ]:
m = RandomForestRegressor(n_estimators=20, min_samples_leaf=10, 
                          n_jobs=8)
%time m.fit(x, y)

In [ ]:
m = RandomForestRegressor(n_estimators=20, min_samples_leaf=3, 
                          n_jobs=8)
%time m.fit(x, y)